## Here I translate GCN in tensorflow to GCN in tensorflow.keras


In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras import layers
import conv_models
import tensorflow.keras.backend as K

In [2]:
# tf and tf keras versions
print(tf.VERSION)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


In [3]:
# load the input data and test set

adj = np.load('/Users/b_eebs/tf-keras/ZINC/adj/0.npy').astype('float32')
features = np.load('/Users/b_eebs/tf-keras/ZINC/features/0.npy').astype('float32')
test = (np.load('/Users/b_eebs/tf-keras/ZINC/logP.npy')[0:10000]).astype('float32') #takes the first 10,000 molecules

In [4]:
# here I define the GCN layers

class convLayers(layers.Layer):
    models = ['gcn', 'gcn_a', 'gcn_g', 'gcn_ag']

    def __init__(self,adjacency,features,output_dim, 
                 weight_shape,bias_shape, model, **kwargs):
        self.output_dim = output_dim 
        self.weight_shape = weight_shape
        self.bias_shape = bias_shape
        self.model = model
        self.features = K.variable(features, 
                                       dtype = tf.float32)
        self.adjacency = K.variable(adjacency, 
                                        dtype = tf.float32)
        super(convLayers, self).__init__(**kwargs)
    
    def compute_output_shape(self, input_shapes):
        #features_shape = input_shapes[0]
        #output_shape = (features_shape[0], self.units)
        output_shape = (100,32)
        return output_shape  # (batch_size, output_dim)
    
    # here are the weight and bias for the layer
    # still have to add attention weights
    # initializers may be incorrect
    def build(self,input_shape):
        self.kernel = self.add_weight(name='kernel',
                                  shape=self.weight_shape,
                                  initializer='glorot_uniform',
                                  trainable=True,dtype=tf.float32)
        
        self.bias = self.add_weight(name='bias',
                                shape=self.bias_shape,
                                initializer='glorot_uniform',
                                trainable=True,dtype=tf.float32)
        super(convLayers, self).build(input_shape)
    
    # here is where I will implement all models, which are built
    # in conv_models.py. The calculation of features will be 
    # used in the dense layer where ReLU will be applied.
    def call(self, inputs):
        if self.model == convLayers.models[0]:
            print('go to gcn function and calc. mult.')
            _features = conv_models.GCN(self.features,self.adjacency,
                                        self.kernel,self.bias)
        elif self.model == convLayers.models[1]:
            print('go to gcn+att function and calc. mult.')
        elif self.model == convLayers.models[2]:
            print('go to gcn+gate function and calc. mult.')
        elif self.model == convLayers.models[3]:
            print('go to aug. gcn function and calc. mult.')
        else:
            raise ValueError('not a valid model')
        return _features

In [12]:
# this is the the compiler and fit for a single molecule with one layer. It works, but isn't complete.
# Need to be able to input more than 1 molecule in batches. The current issue is that the shape of the batched
# input features doesn't talk to the size below I attempt to do the same thing w/o batching but for several layers.
# tried to set batch_size = 100 
conv_output = 32
weight_shape = [features.shape[2],conv_output]
K.set_learning_phase(1)
# define the model, composed of one layer defined in convLayers class
model = tf.keras.Sequential([convLayers(adj[0:100],features[0:100],
                            conv_output,weight_shape,[conv_output],'gcn'), 
                            layers.Activation('relu')])

# model compiler that optimizes and monters the mse and mae per batch
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='mse',
              metrics=['mae'])

# originally sized ((10000, 50, 58) , (10000,))
model.fit(features[0:100], test[0:100], batch_size = 100)

go to gcn function and calc. mult.


InvalidArgumentError: Incompatible shapes: [100,50,32] vs. [100,1]
	 [[{{node training_14/Adam/gradients/loss_7/output_1_loss/MeanSquaredError/sub_grad/BroadcastGradientArgs}}]]

In [13]:
# here i am attempting to add more layers to the model, and test the fit. The issue is that the shape of the 
# features needs to change after the first convLayer, since the dimensions of the weight are changing.

conv_output = 32
weight_shape = [features.shape[2],conv_output]
input_dim = features.shape[2]
K.set_learning_phase(1)
num_layers = 6
dims = [input_dim]
for i in range(num_layers):
    dims.append(32)

layers = []
for i in range(len(dims)-1):
    layers.append((convLayers(adj[0:1],features[0:1],
                    conv_output,[dims[i],dims[i+1]],[32],'gcn')))

model = tf.keras.Sequential(layers)

model.fit(features[0:1], test[0:1],batch_size = 100)

go to gcn function and calc. mult.
go to gcn function and calc. mult.


ValueError: Cannot reshape a tensor with 1024 elements to shape [58,32] (1856 elements) for 'einsum_9/Reshape_1' (op: 'Reshape') with input shapes: [32,32], [2] and with input tensors computed as partial shapes: input[1] = [58,32].